In [1]:
from notebooks.bootstrap_notebooks import bootstrap_notebook, load_dataset, train, test
import pytorch_lightning as pl
import torch
import torch.functional as F
from pl_bolts.models.autoencoders import VAE

from src.base.datasets.sampling_rules.balance import balanced_weight
from src.base.training.models.aes_utils import get_enc_dec
from torch.nn import Flatten, Unflatten, Sequential, Linear, ReLU
from torch.nn.functional import log_softmax, nll_loss
from torchmetrics import Accuracy, Precision, Recall, F1Score, MetricCollection

import torchmetrics

C:\Users\micdu\Code\pythonProject\dmtl\venv\lib\site-packages\pl_bolts\callbacks\data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
C:\Users\micdu\Code\pythonProject\dmtl\venv\lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
C:\Users\micdu\Code\pyth

In [2]:
EXP_NAME = "fraud_classifier"

config = {
    "exp_name": EXP_NAME,
    "enable_gpu": True,
    "training_n_dev": 1,
    "logs": "C:\\Users\\micdu\\Code\\pythonProject\\dmtl\\notebooks\\logs",
    "training_out": "C:\\Users\\micdu\\Code\\pythonProject\\dmtl\\notebooks\\lightning_data",
    "tracking_uri": "http://localhost:5000"
}
bootstrap_notebook(config)

In [3]:
load_dataset({
    "dataset": "credit_card_fraud",
    "data_path": "C:\\Users\\micdu\\Code\\pythonProject\\dmtl\\data",
}, lambda sampler: sampler.set_sampling_rules([balanced_weight()]))

In [4]:



class FraudDetection(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = Linear(30, 10)
        self.a1 = ReLU()
        self.l2 = Linear(10,2)
        metrics = MetricCollection([
            # Accuracy: due to mode multiclass, not multilabel, this uses same formula as Precision
            Accuracy(num_classes=2),
            Precision(num_classes=2),
            Recall(num_classes=2),
            F1Score(num_classes=2),
        ])

        self.train_metrics = metrics.clone(prefix='train_')
        self.valid_metrics = metrics.clone(prefix='val_')
        self.test_metrics = metrics.clone(prefix='test_')

    def forward(self, x):
        x = self.l1(x)
        x = self.l2(self.a1(x))
        return log_softmax(x, dim=1)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1)
        return [optimizer], [lr_scheduler]

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self(x)
        loss = nll_loss(logits, y)
        self.log("training_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nll_loss(logits, y)
        self.log("validation_loss", loss, prog_bar=True)

        preds = torch.argmax(logits, dim=1)
        output = self.train_metrics(preds, y)
        self.log_dict(output)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nll_loss(logits, y)
        self.log("test_loss", loss, prog_bar=True)
        preds = torch.argmax(logits, dim=1)

        output = self.test_metrics(preds, y)
        self.log_dict(output)

In [5]:
model = FraudDetection()
train(model, epochs=3)

Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | l1            | Linear           | 310   
1 | a1            | ReLU             | 0     
2 | l2            | Linear           | 22    
3 | train_metrics | MetricCollection | 0     
4 | valid_metrics | MetricCollection | 0     
5 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
332       Trainable params
0         Non-trainable params
332       Total params
0.001     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\micdu\Code\pythonProject\dmtl\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
C:\Users\micdu\Code\pythonProject\dmtl\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [6]:
test(model)

Auto select gpus: [0]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\micdu\Code\pythonProject\dmtl\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_Accuracy         0.9233173131942749
      test_F1Score          0.9233173131942749
     test_Precision         0.9233173131942749
       test_Recall          0.9233173131942749
        test_loss           0.6217462420463562
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
